# syntax tools




## locating libraries



### retrieving and framework documentation



### creating libraries from code blocks

Code analysis to combine blocks into modules


### manipulating code libraries

Show git repo organization options?

Warn of changes made outside of the module or feature branch?



## code trees

### walk code trees?

Select code from tree?

Use CSS selectors to find code.



In [ ]:

function walkAst() {
    // TODO: adds a feature for parsing default parameters
    // TODO: add a feature for parsing function bodies as the search path
    
}


function selectAST() {
    // TODO: call selectTree after loading code
    // adds a feature for matching code
    //   and automatically return evaluation on another level of arrays
}


 ### Generate CSS selectors from code?
 
 XPath minimizing service from recording prototype app.
 
 TODO: improve this with trees.
 

In [ ]:

static convertXPathToCss(path: string): string {
    // if matched xpath is simple enough, convert it to CSS
    // DIV[contains(@class, "product-tile")]/parent::*/DIV[2]//MD-CARD[contains(@class, "mat-card")]/parent::*/MD-CARD[1]
    return path
            .replace(/\/([a-z-]+)\[@id="(.*?)"]/ig, '/$1#$2')
            .replace(/\/([^\/]+)\[contains\(@class, "(.*?)"\)]/ig, '/$1.$2')
            .replace(/\/parent::[a-z-]+\/[a-z-]+\[([0-9]+)]/ig, ':nth-child($1)')
            .replace(/^\/\//ig, '')
            .replace(/\/\//ig, ' ')
            .replace(/\//ig, ' > ');
}

elementToXPath(path: Array<EventTarget>): string {

    // algorithm magic!
    // create an very specific XPath following these rules
    const addSegment = (elem: Node): string => {
        if (typeof elem === 'undefined' || elem === null) {
            return '/';
        }
        // add:
        // * element tag name,
        const tag = elem.nodeName;
        // * ids,
        let id: string | boolean = false;
        if (typeof (elem as any).getAttribute === 'function') {
            id = (elem as any).getAttribute('id');
        }
        // * classes with nouns (extra credit dictionary or service integration?),
        let className: string | boolean = false;
        if (typeof (elem as any).classList !== 'undefined') {
            className = (elem as any).classList[ 0 ];
        }
        // * indexes when all siblings are homogeneous, i.e. <li> and <tr><td>
        let countTypes = 0;
        if (typeof elem.parentElement !== 'undefined' && elem.parentElement !== null) {
            for (let e = 0; e < elem.parentElement.children.length; e++) {
                const el = elem.parentElement.children[ e ];
                if (el.nodeName === elem.nodeName) {
                    countTypes += 1;
                }
                if (el === elem) {
                    break;
                }
            }
        }

        return `${tag
                ? tag : ''}${id
                ? `[@id="${id}"]` : ''}${className
                ? `[contains(@class, "${className}")]` : ''}${countTypes > 0
                ? `/parent::${elem.parentElement.nodeName}/${tag}[${countTypes}]` : ''}`;
    };
    // use a stack to start for easy debugging
    const pathStack = [];
    // as we traverse parents of evt.target DOM element
    for (const currentEl of path) {
        pathStack.unshift(addSegment(currentEl as Node));
        if ((currentEl as Node).nodeName === 'BODY') {
            break;
        }
        if ((currentEl as Node).nodeName === 'APP-ROOT') {
            return;
        }
    }

    // flatten the XPath stack using '/'
    const fullPath = '//*[not(./app-root)]//' + pathStack.join('/');
    const matches = RecordingComponent.getArrayXPath(fullPath);
    if (matches.length !== 1) {
        throw new Error('Can\'t do anything right!');
    }

    // minimize XPath by splitting up every combination and checking only 1 element is matched for click
    const combinations: Array<string> = [];
    // * between 1 to N segments
    for (let i = 1; i < pathStack.length; i++) {
        for (let j = 1; j < i; j++) {
            const segments = ([
                ...pathStack.slice(j, i),
                // replace match with an extra / to create // between unnecessary segments
                '', // added to create // between target element
                pathStack[ pathStack.length - 1 ]
            ]).join('/');
            combinations.push('//' + segments);
        }
        // * starts from 0 to N
        // *
    }

    // sort by smallest and return shortest path matching 1 DOM element
    const minimal = combinations
            .filter(c => RecordingComponent.getArrayXPath(c).length === 1)
            .sort((a, b) => a.length - b.length);

    return minimal[ 0 ];
    // TODO: prioritize by button/input, ids, classes, attributes? (extra credit), index
}

### generate unit tests

TODO: use language server for this

generate unit test from logic branching?


In [ ]:
var acorn = require("acorn");
var ASTQ  = require("astq");
var _ = require('lodash');
var importer = require('../Core');

var source = importer.interpret('zuora eloqua mapper').code;
    
var comments = [], tokens = [];

var ast = acorn.parse(source, {
    ecmaVersion: 6,
    // collect ranges for each node
	ranges: true,
	// collect comments in Esprima's format
	onComment: comments,
	// collect token ranges
	onToken: tokens,
    
})

var astq = new ASTQ();
var assignments = astq.query(ast, `
//AssignmentExpression [
    //Identifier[@name=='record']
]
`);
var grouped = _.groupBy(assignments, a => comments.filter(c => c.start < a.start).pop().value.trim());
var output = '';
Object.keys(grouped).forEach(k => {
    const ass = grouped[k].map(as => {
        const leftMembers = as.left.type === 'MemberExpression'
            ? [as.left.property.value] : astq.query(as.left, `
//MemberExpression
`).map(m => m.property.value).filter(m => m);
        const rightMembers = as.right.type === 'MemberExpression'
            ? [as.right.property.value] : astq.query(as.right, `
//MemberExpression
`).map(m => m.property.value).filter(m => m);
        if(leftMembers.length === 0 || rightMembers.length === 0) {
            return;
        }
        return `
            it('should map ${leftMembers.join(', ')} to eloqua given ${rightMembers.join(', ')} from zuora', () => {
                const mapped = mapDataToFields(records);
                assert.equal(records[0]['${rightMembers.join('\'] || records[0][\'')}'], mapped[0]['${leftMembers}']);
            })
`;
    }).filter(m => m).join('\n');
    output += `
        describe('${k} properties mapped', () => {
            ${ass}
        });
`
})
//console.log(astq.query(ast, `//AssignmentExpression`));
var destination = [].concat(...astq.query(ast, `//AssignmentExpression`)
    .map(as => as.left.type === 'MemberExpression'
            ? [as.left.property.value]
            : astq.query(as.left, `//MemberExpression`)
         .map(m => m.property.value)
         )).filter(m => m)
var source = astq.query(ast, `
//MemberExpression [
    //Literal
]
`)
    .filter(e => (e.property.value + '').includes('.'))
    .map(e => e.property.value)
console.log(`
var assert = require('assert');
var importer = require('../Core');
var renewalsQuery = importer.import('zuora renewals query');
var { bulkImportTemplate } = importer.import('eloqua create template');
var { getUniqueRatePlans } = importer.import('zuora account blueprints');

var zuoraQuery = renewalsQuery.getQuery('beginning of November', 'beginning of December').Query;

var accounts = getUniqueRatePlans();
for(var records of accounts) {

    describe('given account with ' + records.map(r => r['ProductRatePlan.Name']), () => {
        ${output}
    })

}

describe('check field map', () => {
    it('should map all fields from zuora query', () => {
        const requireFields = ${JSON.stringify(source, null, 4).replace(/\n/ig, '\n    ')};
        const missing = requireFields.filter(f => !zuoraQuery.includes(f));
        assert(missing.length === 0, 'missing fields from zuora query ' + missing.join(', '));
    })

    it('should map all fields in eloqua import definition', () => {
        const mappedFields = ${JSON.stringify(destination, null, 4).replace(/\n/ig, '\n    ')};
        const unmapped = Object.keys(bulkImportTemplate(60).fields).filter(k => !mappedFields.includes(k));
        assert(unmapped.length === 0, 'unmapped eloqua fields ' + unmapped.join(', '));
    })

    it('should map all record fields to eloqua fields', () => {
        const mappedFields = ${JSON.stringify(destination, null, 4).replace(/\n/ig, '\n    ')};
        const missing = mappedFields.filter(f => !Object.keys(bulkImportTemplate(60).fields).includes(f));
        assert(missing.length === 0, 'missing fields from eloqua import definition ' + missing.join(', '));
    })
})

`);


### detecting framework features

Show router tree, module import tree, call tree, feature tree


### format detection by example

Lint all code

Reformat all code like when using webstorm


### common tree queries

TODO: remove all of these and replace with new transpiler


common astq and esquery for notebooks?

common ast queries?


In [ ]:
var GET_CORE = `/VariableDeclaration[//Literal[index(@value, 'Core') > 0]]`;
var GET_CORENAME = `/*/VariableDeclarator/Identifier`;
var GET_DECLARATIONS = `//*[@type == 'VariableDeclarator'
|| @type == 'FunctionDeclaration'
|| @type == 'StaticMemberExpression']`;
var GET_DECLARED = fnName => `${GET_DECLARATIONS}/Identifier[@name == '${fnName}']`;
var GET_CALLED = fnName => `//CallExpression/Identifier[@name == '${fnName}']`;
var GET_IMPORTER = importer => `/*[(@type == 'StaticMemberExpression' || @type == 'MemberExpression')
                && /Identifier[@name == '${importer}']]`;
var GET_CALLER = importer => `//CallExpression[${GET_IMPORTER(importer)}]`;
var GET_PARENT = of => `//*[${of}]`;
// TODO: use form code analysis
var GET_IMPORT_IDENTIFIER = `/Identifier[@name=='import' || @name=='importNotebook']`;
var GET_IMPORT = match => `//CallExpression[
    (${GET_IMPORT_IDENTIFIER}
    || /StaticMemberExpression${GET_IMPORT_IDENTIFIER})
    && //Literal[@value == '${match}']]`;
var GET_EXPORTS = `//AssignmentExpression[//Identifier[@name=='exports']]`;
var GET_FUNCTIONS = `//FunctionDeclaration`;
var GET_IMPORTS = `//CallExpression[${GET_IMPORT_IDENTIFIER} 
    || /StaticMemberExpression${GET_IMPORT_IDENTIFIER}]//Literal`;
var GET_IMPORTER_CALLS = `//CallExpression//Identifier[
    ${[ 'import',
  'streamJson',
  'regexToArray',
  'getCells',
  'runAllPromises',
  'interpret',
  'runInNewContext',
  'cacheCells',
  'interpretMarkdown',
  'fuseSearch',
  'cacheAll' ].map(k => `@name=='${k}'`).join(' || ')}]`
var GET_PARAMETERS = fnName => `//FunctionDeclaration[
    /Identifier[@name == '${fnName}']
]/*[@type != 'BlockStatement']//Identifier`;
var GET_REQUIRE = `//CallExpression[/Identifier[@name=='require']]//Literal`;

module.exports = {
    GET_CORE,
    GET_CORENAME,
    GET_DECLARATIONS,
    GET_DECLARED,
    GET_CALLED,
    GET_IMPORTER,
    GET_CALLER,
    GET_PARENT,
    GET_IMPORT_IDENTIFIER,
    GET_IMPORT,
    GET_EXPORTS,
    GET_FUNCTIONS,
    GET_IMPORTS,
    GET_IMPORTER_CALLS,
    GET_PARAMETERS,
    GET_REQUIRE
}


get ast path array?

TODO: make this act more like getArrayXPath but without dependence on astq, esquery, and esprima




In [ ]:
var ASTQ  = require("astq");
var astq = new ASTQ();
var esquery = require('esquery');
var esprima = require('esprima'); 

function walkAst() {
    // TODO: adds a feature for functions
}


function selectAst() {
    // TODO: call selectTree after loading code
    // adds a feature for matching code
    //   and automatically return evaluation on another level of arrays
}

function getArrayAST(selector, ctx, index) {
    if(typeof ctx === 'string') {
        ctx = esprima.parse(ctx, {range: true, tokens: true,
                                  comment: true, whitespace: true});
    }
    try {
        return astq.query(ctx, selector);
    } catch (e) {
        if(e.message.includes('query parsing failed')) {
            return esquery(ctx, selector);
        } else {
            throw e;
        }
    }
}

// TODO: implement getObjectXPath with ASTs

module.exports = getArrayAST;



test ast path array?

In [ ]:
var importer = require('../Core');
var getArrayAST = importer.import('get ast path array');

function testASTArray(selector, search = '') {
    return getArrayAST(selector, importer.interpret(search).code);
}

module.exports = testASTArray;



get module exports from source?

get exports from source?


In [ ]:
var importer = require('../Core');
var getArrayAST = importer.import('get ast path array');
var {GET_EXPORTS, GET_FUNCTIONS} = importer.import('common ast queries');

// TODO: does this work on other node_modules?
function getExports(code) {
    // can't call it exports or it might detect itself
    var moduleOutput = getArrayAST(GET_EXPORTS, code)
    var exportFunctions = [].concat.apply([], moduleOutput
                                          .map(e => getArrayAST('/Identifier', e)
                                               .map(e => e.name)));
    var outputFunctions = getArrayAST(GET_FUNCTIONS, code)
        .filter(f => exportFunctions.includes(f.id.name))
        .map(f => f.id.name)
    var assignments = moduleOutput
        .map(e => getArrayAST('/StaticMemberExpression', e)
             .map(e => e.property.name))
    return outputFunctions.concat.apply(outputFunctions, assignments);
}

module.exports = getExports;


get parameter names?

function parameters?


In [ ]:
var importer = require('../Core');
var getExports = importer.import('get module exports');
var getArrayAST = importer.import('get ast path array');
var {GET_PARAMETERS, GET_FUNCTIONS} = importer.import('common ast queries');

// TODO: fix calls to this and remove interpret
function getParameters(search) {
    const result = typeof search === 'function'
        ? search.toString()
        : typeof search === 'string'
            ? importer.interpret(search).code
            : search;
    
    var fnName = getExports(result)[0]
        || getArrayAST(GET_FUNCTIONS, result).map(f => f.id.name)[0];
    if(!fnName) {
        return [];
    }
    
    var parameters = getArrayAST(GET_PARAMETERS(fnName), result).map(p => p.name);
    return [fnName, ...(parameters || [])];
}

module.exports = getParameters;


get imports from source?

In [ ]:
var importer = require('../Core');
var importerCode = importer.interpret('import.ipynb');
var getExports = importer.import('get module exports');
var getArrayAST = importer.import('get ast path array');
var {GET_IMPORTS, GET_IMPORTER_CALLS} = importer.import('common ast queries');

function getImports(code) {
    var imports = getArrayAST(GET_IMPORTS, code)
        .map(s => s.value);
    var coreMatches = getArrayAST(GET_IMPORTER_CALLS, code)
        .map(s => s.name)
        .map(i => importerCode
             .filter(core => (core.exports = core.exports || getExports(core.code))
                     .includes(i))[0].id);
    return imports.concat(coreMatches);
}

module.exports = getImports;



get requires from source?

In [ ]:
var importer = require('../Core');
var getArrayAST = importer.import('get ast path array');
var {GET_REQUIRE} = importer.import('common ast queries');

function getRequires(code) {
    return getArrayAST(GET_REQUIRE, code)
        .map(s => s.value)
}


module.exports = getRequires;



get all ast properties?

In [ ]:
var importer = require('../Core');
var getArrayAST = importer.import('get ast path array');
var getExports = importer.import('get exports from source');
var getParameters = importer.import('function parameters');
var getImports = importer.import('get imports from source');

function addAST(cell) {
    cell = typeof cell === 'string'
            ? importer.interpret(cell)
            : cell;
    var code = cell.code || cell;
    try {
        cell.ast = getArrayAST('*', code);
    } catch (e) {
        return cell;
    }
    return Object.assign(cell, {
        exports: getExports(cell.ast),
        params: getParameters(cell.ast),
        imports: getImports(cell.ast)
    });
}

module.exports = addAST;

